Basic CNN

In [ ]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

TensorFlow version: 2.2.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is using GPU? True


In [ ]:
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
train = pd.read_csv('Data/train_new.csv')
train.head()

,image,class
0,Child_p55.mp4_frame20.jpg,1
1,Child_a36.mp4_frame24.jpg,1
2,Child_a49.mp4_frame19.jpg,1
3,Adult_a33.mp4_frame3.jpg,0
4,Adult_p47.mp4_frame35.jpg,0


In [ ]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255.0
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 7820/7820 [02:56<00:00, 44.38it/s]


(7820, 224, 224, 3)

In [ ]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [ ]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense

def create_model():
    
    def add_conv_block(model, num_filters):
        
        model.add(Conv2D(num_filters, 3, activation='relu', padding='same'))
        model.add(BatchNormalization())
        model.add(Conv2D(num_filters, 3, activation='relu', padding='valid'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.2))

        return model
    
    model = tf.keras.models.Sequential()
    model.add(Input(shape=(224, 224, 3)))
    
    model = add_conv_block(model, 32)
    model = add_conv_block(model, 64)
    model = add_conv_block(model, 128)
    model = add_conv_block(model, 256)

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 111, 111, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 111, 111, 64)      2

In [ ]:
%%time

h = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200, batch_size=128
    )

Epoch 1/200
49/49 [==============================] - 20s 413ms/step - loss: 3.8317 - accuracy: 0.5870 - val_loss: 0.6902 - val_accuracy: 0.5377
Epoch 2/200
49/49 [==============================] - 17s 353ms/step - loss: 0.5922 - accuracy: 0.6865 - val_loss: 0.6902 - val_accuracy: 0.5377
Epoch 3/200
49/49 [==============================] - 17s 352ms/step - loss: 0.5736 - accuracy: 0.7030 - val_loss: 0.6965 - val_accuracy: 0.5377
Epoch 4/200
49/49 [==============================] - 17s 353ms/step - loss: 0.5515 - accuracy: 0.7195 - val_loss: 0.6905 - val_accuracy: 0.5377
Epoch 5/200
49/49 [==============================] - 17s 353ms/step - loss: 0.4787 - accuracy: 0.7749 - val_loss: 0.6899 - val_accuracy: 0.5377
Epoch 6/200
49/49 [==============================] - 17s 352ms/step - loss: 0.4449 - accuracy: 0.8272 - val_loss: 0.7000 - val_accuracy: 0.5441
Epoch 7/200
49/49 [==============================] - 17s 353ms/step - loss: 0.3134 - accuracy: 0.8728 - val_loss: 0.6869 - val_accuracy:

In [ ]:
# getting the test list
f = open("test.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,Child/Child_a08.mp4
1,Child/Child_a17.mp4
2,Child/Child_a19.mp4
3,Child/Child_a23.mp4
4,Child/Child_a26.mp4


In [ ]:
# creating the tags
train = pd.read_csv('Data/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [ ]:
from glob import glob
from scipy import stats as s
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('Data/'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % 10 == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255.0
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    #prediction_images = model.predict(prediction_images)
    # converting features in one dimensional array
    #prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    if (videoFile.split('/')[1].split('_')[0] == 'Child'):
        actual.append(1)
    else:
        actual.append(0)
    # actual.append(videoFile.split('/')[1].split('_')[1])

  0%|          | 0/64 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████| 64/64 [03:12<00:00,  3.00s/it]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

90.625